# Collecte des données pour le suivi de trajectoire 

Vous êtes maintenant habitué aux opérations suivantes :
1.  Créer un dataset en collectant des données
2.  Entrainer un modèle
3.  Déployer un modèle

Dans ce notebook, nous allons suivre exactement la même procédure, sauf qu'au lieu de faire de la classification nous allons faire de la **régression**. Cela va nous permettre de faire un suivi de trajectoire avec le JetBot.

Pour collecter les données nécessaires à l'entrainement de notre modèle, vous allez suivre les étapes suivantes :

1. Placer le Jetbot sur différents endroits du chemin à suivre (et ajouter un petit décalage, tester différents angles, etc.)

>  Souvenez-vous de ce qu'on a déjà vu : Les données sont la clé du succès !

2. Afficher le flux vidéo de la caméra.
3. Sélectionner sur l'image un point qui correspond à la cible à suivre.
4. Sauvegarder les coordonnées X, Y de ce point avec l'image correspondante.


Ensuite, dans le notebook dédié à l'entrainement du modèle, nous ferons en sorte que notre réseau de neurones soit capable de prédire les valeurs des coordonnées X, Y du chemin à suivre à partir d'une image de la route.

Nous pourrons alors réaliser une démonstration en temps réel du suivi de trajectoire. Nous utiliserons les prédictions sur les coordonnées X,Y afin de calculer la valeur à mettre dans les moteurs pour faire tourner et avancer le robot.

Voici un petit guide pour vous aider à sélectionner le point cible sur les images :

1.  Observer la vidéo de la caméra
2.  Imaginez le chemin que le robot devrait suivre tout en évitant que le robot ne sorte de la route
3.  Placer le point cible aussi loin que le robot puisse aller sur ce chemin en ligne droite, et de sorte qu'il reste sur la route

> Par exemple, sur une route en ligne droite, le point cible peut être placé à l'horizon. S'il y a des virages, il faudra le placer plus près pour éviter que le robot sorte de la route.

Ce que nous allons faire est de donner une carotte à notre robot, et le robot va suivre cette carotte !

Pour le circuit, vous pouvez par exemple utiliser ce PDF : https://github.com/AlexandreBourrieau/Jetbot/raw/main/Printable%20PDF%20Racetrack%20-%20Modarri%20-%20The%20Ultimate%20Toy%20Car.pdf

### Chargement des librairies

Commençons par charger quelques librairies dont nous aurons besoin, comme par exemple OpenCV qui nous servira à visualiser et sauvegarder les images. Les librairies uuid et datetime sont utilisées pour nommer les fichiers.

In [1]:
# Librairies IPython pour les widgets
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import tensorflow as tf

# Librairie du JetBot pour commander le robot et la caméra
from jetbot import Robot, Camera, bgr8_to_jpeg

# Librairies basiques en Python pour nommer les fichiers
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

### Collecte des données

Affichons le flux de la caméra mais cette fois-ci en utilisant un widget de type `jupyter_clickable_image_widget` qui permet à l'utilisateur de cliquer sur l'image et d'extraire les coordonnées du point cliqué. Ces coordonnées nous servirons à nommer les fichiers.

On utilise la classe Camera du Jetbot pour utiliser la caméra embarquée CSI. Le réseau prend des images au format 224x224 pixels en entrée. On utilise donc cette résolution pour initialiser la caméra.

Le code suivant va afficher le flux vidéo ainsi que la dernière image sauvegardée (avec un cercle vert sur l'endroit cliqué) sur la droite. On affiche également le nombre d'images sauvegardées. Essayez d'en prendre au moins 150.

Lors d'un clic sur une image, le fichier sauvegardé est enregistré dans le répertoire ``dataset_xy`` et nommé de la manière suivante :

``xy_<valeur_X>_<valeur_Y>_<uuid>.jpg``

Lors de l'entrainement, les images seront chargées et les coordonnées X,Y extraites à partir du nom des fichiers. Les valeurs `<valeur_X>` et `<valeur_Y>` sont les coordonnées **en pixels** (calculées depuis le coin supérieur gauche de l'image).



In [2]:
from jupyter_clickable_image_widget import ClickableImageWidget

REPERTOIRE = 'dataset_xy'

# Création du répertoire
try:
    os.makedirs(REPERTOIRE)
except FileExistsError:
    print('Répertoire déjà existant')

camera = Camera()

# Visualisation du flux vidéo et de l'image capturée
widget_camera = ClickableImageWidget(width=camera.width, height=camera.height)
image_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (widget_camera, 'value'), transform=bgr8_to_jpeg)

# Création des widgets
widget_compteur = ipywidgets.IntText(description='compteur')

# Initialise le compteur
widget_compteur.value = len(glob.glob(os.path.join(REPERTOIRE, '*.jpg')))

def sauvegarde_image(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']                           # Coordonnée X
        y = data['offsetY']                           # Coordonnée Y
        
        # Sauvegarde de l'image sur la carte SD
        uuid = 'xy_%03d_%03d_%s' % (x, y, uuid1())
        repertoire_image = os.path.join(REPERTOIRE, uuid + '.jpg')
        with open(repertoire_image, 'wb') as f:
            f.write(widget_camera.value)
        
        # Affiche de l'image sauvegardée en y ajoutant
        # Un petit cercle vert sur le point cliqué
        image = camera.value.copy()
        image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
        image_widget.value = bgr8_to_jpeg(image)
        widget_compteur.value = len(glob.glob(os.path.join(REPERTOIRE, '*.jpg')))
        
widget_camera.on_msg(sauvegarde_image)

widget_global = ipywidgets.VBox([
    ipywidgets.HBox([widget_camera, image_widget]),
    widget_compteur
])

display(widget_global)

Répertoire déjà existant


In [ ]:
camera.stop()

In [ ]:
!zip -r dataset.zip dataset_xy

# Vérification des données enregistrées

In [ ]:
# Librairies IPython pour les widgets
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Librairie du JetBot pour commander le robot et la caméra
from jetbot import Robot, Camera, bgr8_to_jpeg

# Librairies basiques en Python pour nommer les fichiers
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

In [ ]:
REPERTOIRE = 'dataset_xy'

# Création du répertoire
try:
    os.makedirs(REPERTOIRE)
except FileExistsError:
    print('Répertoire déjà existant')

In [ ]:
from jupyter_clickable_image_widget import ClickableImageWidget
import glob

compteur_images = 0

#Récupération des fichiers
repertoires_images = []
repertoire_courant = os.getcwd()
for fichier in glob.glob(repertoire_courant+"/dataset_xy/*.jpg"):
    repertoires_images.append(fichier)

# Visualisation des images
image_widget = ipywidgets.Image(width=224, height=224)

# Création des widgets
widget_compteur = ipywidgets.IntText(description='compteur')
bouton_suivant = ipywidgets.Button(description="Suivant")
bouton_precedent = ipywidgets.Button(description="Précédent")

bouton_supprimer = ipywidgets.Button(description="Supprimer",button_style='danger')

# Initialise le compteur
widget_compteur.value = len(glob.glob(os.path.join(REPERTOIRE, '*.jpg')))


def Clic_BoutonPrecedent(b):
    global compteur_images
    compteur_images = compteur_images - 2
    if compteur_images < 0:
        compteur_images = 0
    Clic_BoutonSuivant(1)

    
def Clic_BoutonSuivant(b):
    global compteur_images
    
    if compteur_images >= len(repertoires_images):
        return

    # Récupératioon des coordonnées
    element = tf.strings.split(repertoires_images[compteur_images],sep="xy_")
    element = tf.strings.split(element[1],sep="_")
    x0 = (tf.strings.to_number(element[0],out_type=tf.dtypes.float32) - 224/2) / (224/2)
    y0 = (tf.strings.to_number(element[1],out_type=tf.dtypes.float32) - 224/2) / (224/2)
    
    # Calcul des coordonnées en pixel
    xs = np.int(224 * ((x0+1) / 2.0))
    ys = np.int(224 * ((y0+1) / 2.0))

    # Lecture de l'image
    image = cv2.imread(repertoires_images[compteur_images])
    compteur_images = compteur_images + 1
    
    # Tracé de la cible sur l'image
    image = cv2.circle(np.array(image), (xs, ys), 8, (255, 0, 0), 3)
    image = cv2.line(np.array(image),(112,224),(xs,ys),(0,0,255),3)
    image_widget.value = bgr8_to_jpeg(image)
    
    
    # Affcihe le compteur
    widget_compteur.value = compteur_images

def Clic_BoutonSupprimer(b):
    global compteur_images
    global repertoires_images
    os.remove(repertoires_images[compteur_images-1])
    
    #Récupération des fichiers
    repertoires_images = []
    repertoire_courant = os.getcwd()
    for fichier in glob.glob(repertoire_courant+"/dataset_xy/*.jpg"):
        repertoires_images.append(fichier)

    compteur_images = compteur_images - 2
    Clic_BoutonSuivant(1)


# Mise en place des liens
bouton_suivant.on_click(Clic_BoutonSuivant)
bouton_precedent.on_click(Clic_BoutonPrecedent)
bouton_supprimer.on_click(Clic_BoutonSupprimer)

widget_global = ipywidgets.VBox([
    ipywidgets.HBox([image_widget]),
    widget_compteur,bouton_precedent,bouton_suivant,bouton_supprimer
])

display(widget_global)
Clic_BoutonSuivant(1)